In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

# 🎯 Few-Shot Prompting으로 AI 성능 극대화하기

## 📚 개요

**Few-Shot Prompting** 은 AI에게 **몇 가지 예시를 보여주어** 원하는 작업을 더 정확하게 수행하도록 도와주는 강력한 기법입니다! 📖

### 🎭 실생활 비유로 이해하기

**과외 선생님이 학생에게 문제를 가르치는 과정**을 떠올려보세요:

1. **📝 문제 유형 설명**: "이런 문제가 나오면..."
2. **✨ 예시 문제 풀이**: "첫 번째 예시는 이렇게, 두 번째는 저렇게..."
3. **🎯 실전 문제**: "이제 너도 비슷한 문제를 풀어봐!"
4. **🏆 정확한 답변**: 예시를 참고해서 올바른 형태로 답변

Few-Shot Prompting도 정확히 이와 같은 원리입니다! 🚀

### 📋 목차

1. **🔧 환경 설정** - 필요한 도구들 준비하기
2. **📚 FewShotPromptTemplate 기본** - 기본 개념과 사용법
   - 예시 데이터 준비하기
   - 프롬프트 템플릿 구성하기
   - 실제 실행해보기
3. **🎯 Example Selector** - 똑똑한 예시 선택하기
   - 유사성 기반 예시 선택
   - 벡터 데이터베이스 활용하기
4. **💬 FewShotChatMessagePromptTemplate** - 대화형 Few-Shot
   - 채팅 형태의 예시 관리
   - 다양한 작업 유형별 예시

### 💡 Few-Shot이 필요한 이유

- **🎯 정확성 향상**: 예시를 통해 원하는 출력 형태를 명확히 전달
- **🔄 일관성 확보**: 항상 같은 형식으로 결과 생성
- **⚡ 학습 효과**: AI가 패턴을 빠르게 이해하고 적용
- **🛠️ 복잡한 작업**: 단순 지시만으로는 어려운 작업도 가능

이제 실제 예제를 통해 Few-Shot Prompting의 강력함을 체험해봅시다! 🎉

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# LLM 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0: 일관된 답변, 1: 창의적 답변)
    model_name="gpt-4.1",  # 사용할 모델명
)

# 간단한 질문으로 LLM 테스트
question = "대한민국의 수도는 뭐야?"

# 질의 실행
answer = llm.stream(question)
stream_response(answer)

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Few-Shot에 사용할 예시 데이터 정의 (복잡한 추론 과정을 단계별로 보여주는 예시들)
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [ ]:
# 개별 예시를 위한 프롬프트 템플릿 생성
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# 첫 번째 예시로 템플릿 테스트
print(example_prompt.format(**examples[0]))

In [ ]:
# FewShotPromptTemplate 생성 (모든 예시를 포함)
prompt = FewShotPromptTemplate(
    examples=examples,  # 사용할 예시 리스트
    example_prompt=example_prompt,  # 개별 예시의 형식
    suffix="Question:\n{question}\nAnswer:",  # 실제 질문이 들어갈 부분
    input_variables=["question"],  # 입력 변수 정의
)

# 새로운 질문으로 프롬프트 생성
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

In [ ]:
# LLM으로 Few-Shot 프롬프트 실행
answer = llm.stream(final_prompt)
stream_response(answer)

In [ ]:
# FewShotPromptTemplate으로 LangChain chain 구성
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Chain 생성: 프롬프트 → LLM → 출력 파서
chain = prompt | llm | StrOutputParser()

# Chain으로 질문 실행
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

---

## 🎯 Example Selector - 똑똑한 예시 선택기

예시가 많아질수록 **"어떤 예시를 선택할까?"** 라는 고민이 생깁니다. 모든 예시를 다 포함하면 프롬프트가 너무 길어지고, 관련 없는 예시까지 포함되면 AI가 혼란스러워할 수 있습니다! 😅

### 🏪 쇼핑몰 상품 추천으로 이해하기

온라인 쇼핑몰에서 **상품 추천**을 생각해보세요:

- **🔍 검색 키워드**: "스니커즈" 
- **📦 전체 상품**: 10,000개의 신발들
- **🎯 추천 결과**: 가장 관련성 높은 10개만 보여줌
- **⭐ 선택 기준**: 가격, 평점, 유사성 등을 종합 고려

**Example Selector** 도 정확히 이와 같은 원리입니다! 전체 예시 중에서 현재 질문과 **가장 유사한 예시들**만 골라서 프롬프트에 포함시켜 줍니다.

### 🧠 Example Selector의 작동 원리

1. **📝 전체 예시 저장**: 미리 준비한 모든 예시들을 벡터 데이터베이스에 저장
2. **🔍 질문 분석**: 사용자의 새로운 질문을 벡터로 변환
3. **📊 유사성 계산**: 질문과 각 예시들 간의 유사도를 수치로 계산
4. **🏆 상위 k개 선택**: 가장 유사한 k개의 예시만 선별
5. **🚀 프롬프트 생성**: 선별된 예시들로 최적화된 프롬프트 완성

### 💡 왜 Example Selector가 필요할까?

- **⚡ 성능 향상**: 관련성 높은 예시만 사용하여 더 정확한 결과
- **💰 비용 절약**: 불필요한 예시를 제외하여 토큰 사용량 감소  
- **🎯 맞춤형 대응**: 상황에 따라 다른 예시들을 자동 선택
- **🔧 유지보수**: 새로운 예시 추가해도 자동으로 최적 선택

이제 실제로 **SemanticSimilarityExampleSelector** 를 사용해서 똑똑한 예시 선택을 경험해봅시다! 🎉

- [📚 API 문서](https://api.python.langchain.com/en/latest/core/example_selectors.html)

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 벡터 데이터베이스 생성 (예시들을 벡터로 저장하기 위함)
chroma = Chroma("example_selector", OpenAIEmbeddings(model="text-embedding-3-small"))

# 유사성 기반 예시 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록
    examples,
    # 의미적 유사성을 측정하는 임베딩 모델
    OpenAIEmbeddings(model="text-embedding-3-small"),
    # 임베딩을 저장하고 유사성 검색을 수행하는 벡터스토어
    Chroma,
    # 선택할 예시의 개수
    k=1,
)

# 테스트 질문
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 입력과 가장 유사한 예시 선택
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

In [ ]:
# Example Selector를 사용하는 FewShotPromptTemplate 생성
prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 예시 리스트 대신 선택기 사용
    example_prompt=example_prompt,  # 개별 예시의 형식
    suffix="Question:\n{question}\nAnswer:",  # 실제 질문 부분
    input_variables=["question"],  # 입력 변수
)

# 질문으로 프롬프트 생성 (자동으로 유사한 예시 선택됨)
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

### 🔗 Example Selector와 FewShotPromptTemplate 연결하기

위에서 **Example Selector** 가 어떻게 똑똑하게 예시를 선택하는지 확인했습니다! 이제 이 기능을 **FewShotPromptTemplate** 과 연결해서 실제로 활용해봅시다. 🚀

**핵심 포인트** 💡:
- `examples` 매개변수 대신 `example_selector` 를 사용
- 선택된 예시들이 자동으로 프롬프트에 포함됨
- 질문이 바뀔 때마다 **가장 적절한 예시들**이 동적으로 선택됨

In [ ]:
# Example Selector가 적용된 FewShotPromptTemplate으로 체인 생성
prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 동적 예시 선택기
    example_prompt=example_prompt,  # 개별 예시 형식
    suffix="Question:\n{question}\nAnswer:",  # 질문 템플릿
    input_variables=["question"],
)

# 체인 생성 (프롬프트 → LLM)
chain = prompt | llm

In [ ]:
# Example Selector를 사용한 체인 실행
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

---

## 💬 FewShotChatMessagePromptTemplate - 대화형 Few-Shot

지금까지는 **단순한 텍스트 형태**의 Few-Shot을 살펴봤습니다. 하지만 실제 ChatGPT 같은 대화형 AI에서는 **Human(사용자)** 과 **AI(어시스턴트)** 가 주고받는 **대화 형태**로 예시를 제공하는 것이 더 효과적입니다! 💬

### 🎭 연극 대본으로 이해하기

**연극 대본**을 생각해보세요:

```
👤 김철수: "오늘 날씨가 어때?"
🤖 AI 봇: "오늘은 맑고 따뜻한 날씨입니다. 외출하기 좋은 날이에요!"

👤 이영희: "내일 비가 올까?"  
🤖 AI 봇: "내일은 오후부터 가벼운 비가 예상됩니다. 우산을 챙겨주세요!"
```

**FewShotChatMessagePromptTemplate** 은 이런 **대화 형태의 예시들**을 AI에게 보여줘서, 실제 사용자와 대화할 때도 **비슷한 톤과 방식**으로 답변하도록 훈련시킵니다!

### 🔧 채팅 Few-Shot의 특별한 점

#### 일반 FewShotPromptTemplate vs ChatMessage 버전

**📝 일반 버전**:
```
Input: 질문
Output: 답변
```

**💬 ChatMessage 버전**:
```
Human: 질문
AI: 답변
Human: 또 다른 질문  
AI: 또 다른 답변
```

### 🎯 언제 ChatMessage 버전을 쓸까?

- **✅ 권장하는 경우**: 대화형 AI, 챗봇, 상담 시스템 등
- **✅ 특별히 유용한 상황**: 
  - 특정 **말투나 톤**을 유지하고 싶을 때
  - **역할 설정**이 중요한 경우 (상담사, 선생님, 전문가 등)
  - **대화의 맥락**을 고려한 답변이 필요할 때

이제 실제로 다양한 작업 유형별 **대화 예시**를 만들어서 활용해봅시다! 🚀

In [ ]:
# 대화형 Few-Shot에 사용할 다양한 작업 예시들
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 채팅용 벡터 데이터베이스 생성
chroma = Chroma("fewshot_chat", OpenAIEmbeddings(model="text-embedding-3-small"))

# 개별 예시를 위한 채팅 프롬프트 템플릿 (Human-AI 대화 형태)
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),  # 사용자의 지시사항과 입력
        ("ai", "{answer}"),  # AI의 응답
    ]
)

# 유사성 기반 예시 선택기 생성 (instruction 기준으로 유사성 판단)
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록
    examples,
    # 의미적 유사성을 측정하는 임베딩 모델
    OpenAIEmbeddings(model="text-embedding-3-small"),
    # 임베딩을 저장하고 유사성 검색을 수행하는 벡터스토어
    chroma,
    # 선택할 예시의 개수
    k=1,
    # 유사성 판단 기준 (instruction 필드 기준)
    input_keys=["instruction"],
)

# FewShotChatMessagePromptTemplate 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,  # 동적 예시 선택기
    example_prompt=example_prompt,  # 개별 예시 형식
)

### 🎯 Example Selector를 사용한 동적 예시 선택

위에서 **FewShotChatMessagePromptTemplate** 과 **Example Selector** 를 결합했습니다! 이제 사용자의 질문에 따라 **가장 관련성 높은 예시**가 자동으로 선택되어 프롬프트에 포함됩니다.

**작동 과정** 🔄:
1. 사용자가 질문을 입력
2. Example Selector가 질문과 유사한 예시 1개를 자동 선택  
3. 선택된 예시와 함께 프롬프트 생성
4. AI가 예시의 패턴을 참고하여 답변 생성

이런 방식으로 **맥락에 맞는 정확한 답변**을 얻을 수 있습니다! ✨

In [ ]:
# 회의록 작성 관련 질문으로 유사한 예시 선택 테스트
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# instruction을 기준으로 가장 유사한 예시 선택
example_selector.select_examples(question)

In [ ]:
# 최종 채팅 프롬프트 구성: 시스템 메시지 + 예시 + 사용자 질문
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",  # 시스템 역할 정의
        ),
        few_shot_prompt,  # 선택된 예시가 자동으로 포함됨
        ("human", "{instruction}\n{input}"),  # 실제 사용자 질문
    ]
)

In [ ]:
# 대화형 Few-Shot Chain 생성 및 실행
chain = final_prompt | llm

# 회의록 작성 요청으로 chain 실행
answer = chain.stream(question)
stream_response(answer)

In [ ]:
# 다른 종류의 질문으로 예시 선택 테스트
question = {
    "instruction": "회의록을 작성해 주세요",
}

# instruction만으로도 적절한 예시 선택 가능
example_selector.select_examples(question)

In [ ]:
# 완전히 다른 작업(요약) 요청시 예시 선택 결과 확인
example_selector.select_examples({"instruction": "다음 문장을 요약해 주세요"})